In [1]:
suppressPackageStartupMessages({
    library(tidyverse)
    library(ggplot2)
    library(palmerpenguins)
    library(lubridate)
    library(DBI)
    library(RSQLite)
    library(httr2)
    library(rvest)
    library(jsonlite)
})

# REST APIs and Web Scraping

In [2]:
# Define the URL for the API
url <- "https://api.nobelprize.org/2/nobelPrizes?nobelPrizeCategory=phy&format=json"

# Create a request object
request_obj <- request(url)

# Perform the request and fetch the data
response <- request_obj %>%
  req_perform()

response

<httr2_response>

GET https://api.nobelprize.org/2.1/nobelPrizes?nobelPrizeCategory=phy&format=json

Status: 200 OK

Content-Type: application/json

Body: In memory (25712 bytes)



In [3]:
# Parse the response body into a JSON object
response <- response %>%
    resp_body_json(flatten = TRUE)

## Web Scraping

In [8]:
url <- "https://www.worldometers.info/world-population/population-by-country/"

html <- read_html(url)

# Scrape table
population_table <- data.frame(
    id = as.numeric(),
    Country = character(),
    Population = character(),
    Yearly_Change = character(),
    Net_Change = character(),
    Density = character(),
    Land_Area = character(),
    Migrant_net = character()
)

data_list <- list()

table <- html %>%
    html_elements("tbody")

rows <- table %>%
    html_elements("tr")


for (i in seq_along(rows)){
    row <- rows[i]
    row_data <- row %>% 
        html_elements("td") %>% 
        html_text2() %>%
        as.character()

    data_list[[i]] <- row_data
}

# Combine all rows into a data frame
population_table <- do.call(rbind, data_list) %>%
    as.data.frame(stringsAsFactors = FALSE)

In [9]:
population_table

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,India,"1,450,935,791",0.89 %,"12,866,195",488,"2,973,190","-630,830",2.0,28,37 %,17.78 %
2,China,"1,419,321,278",-0.23 %,"-3,263,655",151,"9,388,211","-318,992",1.0,40,66 %,17.39 %
3,United States,"345,426,571",0.57 %,"1,949,236",38,"9,147,420","1,286,132",1.6,38,82 %,4.23 %
4,Indonesia,"283,487,931",0.82 %,"2,297,864",156,"1,811,570","-38,469",2.1,30,59 %,3.47 %
5,Pakistan,"251,269,164",1.52 %,"3,764,669",326,"770,880","-1,401,173",3.5,20,34 %,3.08 %
6,Nigeria,"232,679,478",2.10 %,"4,796,533",255,"910,770","-35,202",4.4,18,54 %,2.85 %
7,Brazil,"211,998,573",0.41 %,"857,844",25,"8,358,140","-225,510",1.6,34,91 %,2.60 %
8,Bangladesh,"173,562,364",1.22 %,"2,095,374","1,333","130,170","-473,362",2.1,26,42 %,2.13 %
9,Russia,"144,820,423",-0.43 %,"-620,077",9,"16,376,870","-178,042",1.5,40,75 %,1.77 %
